<a href="https://colab.research.google.com/github/mr7495/OCT-classification/blob/main/ROCT_Net_OCTID_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [20]:
!pip install git+https://github.com/keras-team/keras-contrib

  Cloning https://github.com/keras-team/keras-contrib to /tmp/pip-req-build-l0d7czvs
  Running command git clone -q https://github.com/keras-team/keras-contrib /tmp/pip-req-build-l0d7czvs
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=ab26a08a4cd74d1525180459a101449ee79854e52c579d7220a44eebfdd60de8
  Stored in directory: /tmp/pip-ephem-wheel-cache-6i96xt16/wheels/8e/09/42/ae2d52e8651acfb0595f0f271e668a85ace2f9eb92022307ab
Successfully built keras-contrib


In [21]:
pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.0 MB/s 


In [22]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os
import tensorflow as tf
from keras_contrib.layers import Capsule
from keras_contrib.activations import squash

In [23]:
!gdown --id "1okQhlbA6RNJ23fQG_7z-zcZEiCfCT8pb" #if this code failed, get the dataset from https://drive.google.com/file/d/1okQhlbA6RNJ23fQG_7z-zcZEiCfCT8pb/view?usp=sharing

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1okQhlbA6RNJ23fQG_7z-zcZEiCfCT8pb 



In [26]:
archive = zipfile.ZipFile('OCTA-gholami.zip') 
for file in archive.namelist():
     archive.extract(file, '.')

In [27]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.1,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1)

test_datagen = keras.preprocessing.image.ImageDataGenerator()

train_df = pd.read_csv("train_gholami.csv")
test_df = pd.read_csv("test_gholami.csv")

In [28]:
#create dataloader
shape=(512, 512)
batch_size=10
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='gholami',
      x_col="filename",
      y_col="class",
      target_size=shape,
      batch_size=batch_size,
      class_mode='categorical',shuffle=True)
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='gholami',
        x_col="filename",
        y_col="class",
        target_size=shape,
        batch_size=batch_size,
        class_mode='categorical',shuffle=True)
train_img_num=len(train_generator.filenames)

Found 459 validated image filenames belonging to 5 classes.
Found 113 validated image filenames belonging to 5 classes.


In [29]:
!git clone https://github.com/mhrahimzadeh1374/automl

Cloning into 'automl'...
remote: Enumerating objects: 3922, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 3922 (delta 193), reused 217 (delta 164), pack-reused 3613
Receiving objects: 100% (3922/3922), 23.18 MiB | 12.78 MiB/s, done.
Resolving deltas: 100% (2945/2945), done.


In [34]:
cd automl/efficientnetv2 

/content/automl/efficientnetv2


In [35]:
from effnetv2_model import get_model

In [38]:
cd ../../

/content


In [ ]:
name="ROCT-Net_OCTID dataset"
keras.backend.clear_session()
input_tensor=keras.Input(shape=(shape[0],shape[1],3))
base_model1=get_model('efficientnetv2-b0', include_top=False, pretrained=True)(input_tensor) #load EfficientNetV2-B0
base_model2=keras.applications.Xception(input_tensor=input_tensor,weights='imagenet',include_top=False)(input_tensor) #load Xception

concatenated=keras.layers.concatenate([base_model1,base_model2]) #load concatenated model

avg=keras.layers.AveragePooling2D(3,padding='valid')(concatenated) #deploy Wise-srNet
depthw=keras.layers.DepthwiseConv2D(5,
                                      depthwise_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                                      bias_initializer=keras.initializers.Zeros(),depthwise_constraint=keras.constraints.NonNeg())(avg)
# Define Capsules
capsule = Capsule(num_capsule=10,
                dim_capsule=16,
                routings=3,
                activation=squash,
                share_weights=True)(depthw)

flat=keras.layers.Flatten()(capsule)
dp=keras.layers.Dropout(0.2)(flat)
preds=keras.layers.Dense(5,activation='softmax',
                          kernel_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                          bias_initializer=keras.initializers.Zeros(),)(dp)
model=keras.Model(inputs=input_tensor, outputs=preds)  

##################################
for layer in model.layers:
  layer.trainable = True
model.summary()
filepath="drive/MyDrive/OCT/models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%name
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max',save_weights_only=True) #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

lr_schedule =keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.045,
    decay_steps=2*int(train_img_num/batch_size),
    decay_rate=0.94,
    staircase=True)
optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

hist=model.fit_generator(train_generator, epochs=40,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training
with open('{}-results.csv'.format(name), mode='w',newline='') as csv_file: #write evaluation metrics
  csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  for key in hist.history:
    data=[key]
    data.extend(hist.history[key])
    csv_writer.writerow(data)